In [1]:
!CUDA_VISIBLE_DEVICES=0,1,2,3

In [2]:
import json, transformers, torch
from torch import bfloat16, cuda
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [3]:
def load_jsonl(filename):
    with open(filename, 'r') as f:
        return [json.loads(line) for line in f]

In [4]:
dataset = load_jsonl('chunked-output.jsonl')
dataset[0:10]

FileNotFoundError: [Errno 2] No such file or directory: 'chunked-output.jsonl'

In [2]:
"""instatiating the model and quantizing config"""

model_name = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype= bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(model_name)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#model.eval()

with open('test-questions.txt', 'r') as f:
    text = f.read()

questions = text.split('\n\n')


batch_size = 5  # Adjust based on your multi-GPU capacity.
batches = [questions[i:i + batch_size] for i in range(0, len(questions), batch_size)]

with open('base-output.txt', 'w') as f:
    for batch in batches: 
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model.generate(**inputs, max_new_tokens=2048)
        for output in outputs:
            f.writelines(tokenizer.decode(output, skip_special_tokens=True))
            f.writelines('\n')
            f.writelines("-----------")
            f.writelines('\n')
    

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
def tokenize_example(example): #function to tokenize a data point
    if (example['heading'] != None and example['content'] != None):
        return tokenizer(example['heading'] + ' ' + example['content'], return_tensors="pt", truncation=True, padding='max_length', max_length=1024)
    
def tokenize_and_squeeze(example):
    tokenized_example = tokenize_example(example)
    if tokenized_example:
        tokenized_example["input_ids"] = tokenized_example["input_ids"].squeeze(0)
        tokenized_example["attention_mask"] = tokenized_example["attention_mask"].squeeze(0)
    return tokenized_example

train_input_ids = [tokenize_and_squeeze(example) for example in train_data]
test_input_ids = [tokenize_and_squeeze(example) for example in test_data]

train_input_ids = [item for item in train_input_ids if item is not None]
test_input_ids = [item for item in test_input_ids if item is not None]

for example in train_input_ids:
    example["labels"] = example["input_ids"].clone()
for example in test_input_ids:
    example["labels"] = example["input_ids"].clone()

for x in train_input_ids:
    print(x['input_ids'], x['labels'])

tensor([    1, 29871, 29946,  ...,     2,     2,     2]) tensor([    1, 29871, 29946,  ...,     2,     2,     2])
tensor([    1, 29871, 29906,  ...,     2,     2,     2]) tensor([    1, 29871, 29906,  ...,     2,     2,     2])
tensor([    1, 29871, 29906,  ...,     2,     2,     2]) tensor([    1, 29871, 29906,  ...,     2,     2,     2])
tensor([    1, 29871, 29906,  ...,     2,     2,     2]) tensor([    1, 29871, 29906,  ...,     2,     2,     2])
tensor([    1, 29871, 29896,  ...,     2,     2,     2]) tensor([    1, 29871, 29896,  ...,     2,     2,     2])
tensor([    1, 29871, 29941,  ...,     2,     2,     2]) tensor([    1, 29871, 29941,  ...,     2,     2,     2])
tensor([    1, 29871, 29896,  ...,     2,     2,     2]) tensor([    1, 29871, 29896,  ...,     2,     2,     2])
tensor([    1, 29871, 29896,  ...,     2,     2,     2]) tensor([    1, 29871, 29896,  ...,     2,     2,     2])
tensor([    1, 29871, 29896,  ...,     2,     2,     2]) tensor([    1, 29871, 29896,  .

In [19]:
# Ensure that none of the items in the dataset are None
train_input_ids = [item for item in train_input_ids if item is not None]
test_input_ids = [item for item in test_input_ids if item is not None]
assert not any(item is None for item in train_input_ids), "Found a None item in train_input_ids"
assert not any(item is None for item in test_input_ids), "Found a None item in test_input_ids"


In [15]:
from datasets import Dataset

train_dataset = Dataset.from_dict({
    'type': [item['type'] for item in dataset],
    'heading': [item['heading'] for item in dataset],
    'content': [item['content'] for item in dataset]
})


train_dataset

Dataset({
    features: ['type', 'heading', 'content'],
    num_rows: 4677
})

In [13]:
from peft import LoraConfig, get_peft_model
from datasets import Dataset


lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

from transformers import TrainingArguments

output_dir = "./results_lora"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="content",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

AttributeError: 'list' object has no attribute 'map'

In [15]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 